<a href="https://colab.research.google.com/github/ayeshasGithub/hello-world/blob/master/Read_plc_tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Kafka depedencies

In [ ]:
#plc program update July21

import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'#'magna_simulation'# magnaPlcTagData
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.Data.LaneID':'line','root.Data.ZoneID':'zone','root.Data.TagID':'tag_name','root.Data.StationID':'station',
                
                        'root.Data.EquipmentID':'asset','root.Data.TagValue':'tag_value','root.Data.TagDateTime':'tag_datetime'}

msg_count=0
#plc_tag_hash={}#dic of dic
plc_tag_msg={}

consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='latest',
                         session_timeout_ms=10000, 
                          enable_auto_commit=False, auto_commit_interval_ms=5000,
                         reconnect_backoff_ms=30000, reconnect_backoff_max_ms=60000,
                         max_poll_records=50, max_poll_interval_ms=300000,                        
                         group_id='plc_group_a', value_deserializer=lambda x: json.loads(x.decode('utf-8')))


for msg in consumer:
    msg_count=msg_count+1
    #if msg_count<= 30000:
    #    continue
    msg = msg.value
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    
    len_value=len(new_dic['VALUES'])
    print('number of tags in this batch',len_value)#CHANGE 1
    
    for i in range(len_value):
        
        value_list=new_dic['VALUES'][i]#type(new_dic['Values'])=list of list\n",
        #value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
        #print(name_list)
        #print(value_list) 
        #make one dic from one parsed msg => tag
        plc_dic={}
        for name,value in zip(name_list,value_list):
            #print(name,':',value)
            if value is None:
                value=str(-1)
            plc_dic[key_conversion[name]]=str(value).lower()
        
        #print(plc_dic)#CHANGE 2
    
        #make primary key for the tag
        pk_tag=plc_dic['line']+'.'+plc_dic['zone']+'.'+plc_dic['station']+'.'+plc_dic['asset']+'.'+plc_dic['tag_name']
        pk_tag=pk_tag.replace('.-1','')
        #print(pk_tag)
    
        #plc_tag_msg[pk_tag]=plc_dic#COMMENT
    
    
        #redis write    
        #r_val=json.dumps(plc_tag_msg)#COMMENT
        #r.set('plc_tag_hash',r_val)#UNCOMMENT
        #r.hset('plc_tag_experiment',pk_tag,json.dumps(plc_dic))#UNCOMMENT
        #r.hset('plc_tag_hash_simu',pk_tag,json.dumps(plc_dic))#UNCOMMENT
        r.hset('plc_tag_hash_real',pk_tag,json.dumps(plc_dic))#UNCOMMENT
    
        #redis read
        '''
        r_had=r.get('plc_tag_hash')
        plc_tag=json.loads(r_had)
        #print('from redis read: ',plc_tag)
        #'''    
    
    
    
        #print(plc_dic)
        #print('tag_name: ',plc_dic['tag_name'])
        #print('tag_value: ',plc_dic['tag_value'])
    
    if msg_count%100==0:
        #pass
        print(msg_count,end=' ')
    '''
    if msg_count==1:#0001:
        print('done')
        break
    #'''
    #break

In [ ]:
#new 3
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
#plc_tag_hash={}#dic of dic
plc_tag_msg={}

consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                         session_timeout_ms=10000, 
                          enable_auto_commit=False, auto_commit_interval_ms=5000,
                         reconnect_backoff_ms=30000, reconnect_backoff_max_ms=60000,
                         max_poll_records=50, max_poll_interval_ms=300000,                        
                         group_id=None, value_deserializer=lambda x: loads(x.decode('utf-8')))


for msg in consumer:
    msg_count=msg_count+1
    #if msg_count<= 30000:
    #    continue
    msg = msg.value
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    
    len_value=len(new_dic['VALUES'])
    print('number of values',len_value)
    
    for i in range(len_value):
        
        value_list=new_dic['VALUES'][i]#type(new_dic['Values'])=list of list\n",
        #value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
    
         
        #make one dic from one parsed msg => tag
        plc_dic={}
        for name,value in zip(name_list,value_list):
            #print(name,':',value)
            if value is None:
                value=str(-1)
            plc_dic[key_conversion[name]]=str(value).lower()
        
        #print(plc_dic)
    
        #make primary key for the tag
        pk_tag=plc_dic['line']+'.'+plc_dic['zone']+'.'+plc_dic['station']+'.'+plc_dic['asset']+'.'+plc_dic['tag_name']
        pk_tag=pk_tag.replace('.-1','')
        #print(pk_tag)
    
        plc_tag_msg[pk_tag]=plc_dic
    
    
        #redis write    
        r_val=json.dumps(plc_tag_msg)
        #r.set('plc_tag_hash',r_val)#UNCOMMENT
        #r.hset('plc_tag_experiment',pk_tag,json.dumps(plc_dic))#UNCOMMENT
    
        #redis read
        '''
        r_had=r.get('plc_tag_hash')
        plc_tag=json.loads(r_had)
        #print('from redis read: ',plc_tag)
        #'''    
    
    
    
        #print(plc_dic)
        print('tag_name: ',plc_dic['tag_name'])
        #print('tag_value: ',plc_dic['tag_value'])
    
    if msg_count%500==0:
        print(msg_count,end=' ')
    if msg_count==3:#0001:
        print('done')
        break
    #break

In [ ]:
#new 4
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magna_simulation'#'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.Data.LaneID':'line','root.Data.ZoneID':'zone','root.Data.TagID':'tag_name','root.Data.StationID':'station',
                
                        'root.Data.EquipmentID':'asset','root.Data.TagValue':'tag_value','root.Data.TagDateTime':'tag_datetime'}

msg_count=0
#plc_tag_hash={}#dic of dic
plc_tag_msg={}

consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='latest',
                         session_timeout_ms=10000, 
                          enable_auto_commit=False, auto_commit_interval_ms=5000,
                         reconnect_backoff_ms=30000, reconnect_backoff_max_ms=60000,
                         max_poll_records=50, max_poll_interval_ms=300000,                        
                         group_id='plc_group_a', value_deserializer=lambda x: loads(x.decode('utf-8')))


for msg in consumer:
    msg_count=msg_count+1
    #if msg_count<= 30000:
    #    continue
    msg = msg.value
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    
    len_value=len(new_dic['VALUES'])
    print('number of values',len_value)
    
    for i in range(len_value):
        
        value_list=new_dic['VALUES'][i]#type(new_dic['Values'])=list of list\n",
        #value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
        #print(name_list)
        #print(value_list) 
        #make one dic from one parsed msg => tag
        plc_dic={}
        for name,value in zip(name_list,value_list):
            #print(name,':',value)
            if value is None:
                value=str(-1)
            plc_dic[key_conversion[name]]=str(value).lower()
        
        #print(plc_dic)
    
        #make primary key for the tag
        pk_tag=plc_dic['line']+'.'+plc_dic['zone']+'.'+plc_dic['station']+'.'+plc_dic['asset']+'.'+plc_dic['tag_name']
        pk_tag=pk_tag.replace('.-1','')
        #print(pk_tag)
    
        #plc_tag_msg[pk_tag]=plc_dic#COMMENT
    
    
        #redis write    
        #r_val=json.dumps(plc_tag_msg)#COMMENT
        #r.set('plc_tag_hash',r_val)#UNCOMMENT
        #r.hset('plc_tag_experiment',pk_tag,json.dumps(plc_dic))#UNCOMMENT
        r.hset('plc_tag_hash_simu',pk_tag,json.dumps(plc_dic))#UNCOMMENT
    
        #redis read
        '''
        r_had=r.get('plc_tag_hash')
        plc_tag=json.loads(r_had)
        #print('from redis read: ',plc_tag)
        #'''    
    
    
    
        #print(plc_dic)
        print('tag_name: ',plc_dic['tag_name'])
        #print('tag_value: ',plc_dic['tag_value'])
    
    if msg_count%500==0:
        print(msg_count,end=' ')
    if msg_count==3:#0001:
        print('done')
        break
    #break

In [ ]:
#redis read for Gema, convert byte string
    
#r_had=r.get('plc_tag_hash')
r_had=r.hgetall('plc_tag_hash_simu')
#r_had=json.loads(r_had)
#plc_tag=json.loads(r_had)
#print('from redis read: ',len(plc_tag))
print(type(r_had))
#dic=dict(r_had)#json.loads(r_had)
#print(dic)

for key,value in r_had.items():
    print('key',str(key,"utf-8"))
    print('value',str(value,"utf-8"))
#print(dic['rw43.zone02.st100.100sr1.cycletimetimeracc'])
#print(r_had['rw43.plc02.st060.equip001.rw43.plc02.st060.equip001.lsrst'])
    #'''    

In [ ]:
#redis experiment

#initialize dic
new_dic={}

new_dic[0]='we'
new_dic['1']='he'

print(new_dic)


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.StrictRedis(host="192.168.2.11", port=6379, db=0)
#'''


key='any'
#redis write    
#r_val=json.dumps(plc_tag_msg)
r.hset('plc_tag_experiment',key,json.dumps(new_dic))#UNCOMMENT
    
#redis read
r_haddy=r.hgetall('plc_tag_experiment')
print('from redis read: ',r_haddy)
#'''    
    

In [ ]:
#Experiment kafka
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


msg_count=0

consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='latest',
                         session_timeout_ms=10000, 
                          enable_auto_commit=False, auto_commit_interval_ms=5000,
                         reconnect_backoff_ms=30000, reconnect_backoff_max_ms=60000,
                         max_poll_records=50, max_poll_interval_ms=300000,
                         
                         group_id=None, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    if msg_count==5:
        print(msg_count)
        break

In [ ]:
#Redis
import the Redis client
import redis


# Create a redis client
redisClient = redis.StrictRedis(host='localhost', port=6379, db=0)

# Add key value pairs to the Redis hash
hashName = "Dessert"
redisClient.hset(hashName, 'line', "Cheesecake")
redisClient.hset(hashName, 'zone', "zone01")
redisClient.hset(hashName, 'station', "-1")
redisClient.hset(hashName, 'asset', "-1")
redisClient.hset(hashName, 'tag_name', "Zone01_Shift_1")
redisClient.hset(hashName, 'tag_value', "1")

# Print the hash
print(redisClient.hgetall(hashName))pl

# Remove a key
redisClient.hdel(hashName, 1)
 

# Print the hash after removing a key
print(redisClient.hgetall(hashName))

{'line': 'rw43',
 'zone': 'zone01',
 'station': '-1',
 'asset': '-1',
 'tag_name': 'Zone01_Shift_1',
 'tag_value': '1'
} 



In [ ]:
#new 1
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
plc_tag_hash={}#dic of dic
plc_tag_msg={}
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    #print('\n')
    sys.stdout.write('%d'%msg_count)
    #print(msg_count)
    sys.stdout.flush()
    #time.sleep(0.1)
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
    #make one dic from one parsed msg => tag
    plc_dic={}
    for name,value in zip(name_list,value_list):
        #print(name,':',value)
        if value is None:
            value=str(-1)
        plc_dic[key_conversion[name]]=str(value).lower()
        
    #print(plc_dic)
    
    #make primary key for the tag
    pk_tag=plc_dic['line']+'.'+plc_dic['zone']+'.'+plc_dic['station']+'.'+plc_dic['asset']+'.'+plc_dic['tag_name']
    pk_tag=pk_tag.replace('.-1','')
    #print(pk_tag)
    
    plc_tag_msg[pk_tag]=plc_dic
    
    
    #redis write    
    r_val=json.dumps(plc_tag_msg)
    r.set('plc_tag_hash',r_val)
    
    #redis read
    r_had=r.get('plc_tag_hash')
    plc_tag=json.loads(r_had)
    #print('from redis read: ',plc_tag)
    #'''    
    
    
    
    #print(plc_dic)
    #print('tag_name: ',plc_dic['tag_name'])
    #print('tag_value: ',plc_dic['tag_value'])
    
    
    if msg_count%100==0:
        print(msg_count,end=' ')
    if msg_count==30000:
        print('done')
        break
    #break

ModuleNotFoundError: ignored

In [ ]:
#new
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    print('\n')
    sys.stdout.write('%d'%msg_count)
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
    #make one dic from one parsed msg => tag
    plc_dic={}
    for name,value in zip(name_list,value_list):
        #print(name,':',value)
        plc_dic[key_conversion[name]]=value
    
    
    #read/write one tag/msg on redis
    r_val=json.dumps(plc_dic)
    r.set('plc_tag_msg',r_val)
    r_had=r.get('plc_tag_msg')
    plc_tag=json.loads(r_had)
    print('from redis read: ',plc_tag)
    
    
    
    
    #print(plc_dic)
    print('tag_name: ',plc_dic['tag_name'])
    print('tag_value: ',plc_dic['tag_value'])
    
  
    if msg_count==15:
        break
    #break

ModuleNotFoundError: ignored

In [ ]:
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          group_id='communication_module',
                          enable_auto_commit=False, 
                          max_poll_records=2, max_poll_interval_ms= 300000, 
                          value_deserializer=lambda x: loads(x.decode('utf-8')))


NameError: ignored

In [ ]:
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    print('\n')
    sys.stdout.write('%d'%msg_count)
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
    #make one dic from one parsed msg => tag
    plc_dic={}
    for name,value in zip(name_list,value_list):
        #print(name,':',value)
        plc_dic[key_conversion[name]]=value
    
    #print(plc_dic)
    print('tag_name: ',plc_dic['tag_name'])
    print('tag_value: ',plc_dic['tag_value'])
  
    if msg_count==15:
        break
    #break

ModuleNotFoundError: ignored

In [ ]:
import json
from kafka import KafkaConsumer
import redis
import time
import sys
    
# kafka host (broker)
topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']

#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                           enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
    
for msg in consumer:
  msg = msg.value
  msg_count=msg_count+1
  sys.stdout.write('\\r%d'%msg_count) 
  print('{}'.format(msg))
  
  #parse tag from msg
  dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
  res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
  new_dic=res[0]#type(new_dic)=dic\n",
  name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
  value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
  #make one dic from one parsed msg => tag
  plc_dic={}
  for name,value in zip(name_list,value_list):
      #print(name,':',value)\n",
      plc_dic[key_conversion[name]]=value
      #print(plc_dic)\n",
      print(plc_dic['tag_name'])
    
      if msg_count==1:
        first_tag=plc_dic['tag_name']
        if plc_dic['tag_name']==first_tag:
          print ('tag repeats')
          print('this_tag',plc_dic['tag_name'])
          print('msg_count',msg_count)
          break
          sys.stdout.flush()
        time.sleep(0.2)
        if msg_count==15:
            break\n",
        #break\n",
    

SyntaxError: ignored

In [ ]:
#new 2
import json 
from kafka import KafkaConsumer
import redis
import time
import sys

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']


#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)

#key conversion: {'line': 'rw43', 'zone': 'zone01', 'station': '-1', 'asset': '-1', 'tag_name': 'Zone01_Shift_1', 
#                   'tag_value': '1', 'tag_datetime':'2020-05-28T11:31:05.89'} 
key_conversion={'root.LaneID':'line','root.ZoneID':'zone','root.TagID':'tag_name','root.StationID':'station',
                
                        'root.EquipmentID':'asset','root.TagValue':'tag_value','root.TagDateTime':'tag_datetime'}

msg_count=0
plc_tag_hash={}#dic of dic
plc_tag_msg={}
consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,auto_offset_reset='earliest',
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for msg in consumer:
    msg_count=msg_count+1
    msg = msg.value
    
    #print('{0}\r'.format(msg_count)),
    #print(msg_count,end=' ',flush=True)
    #sys.stdout.write('%d'%msg_count)
    #print(msg_count)
    #sys.stdout.flush()
    #time.sleep(0.1)
    
    #print('{}'.format(msg))
  
    #parse tag from msg
    dic_msg=msg[0]#type(msg)=list, type(msg[0])=dic\n",
    res=json.loads(dic_msg['FeedData'])#type(dic_msg['FeedData'])=str, type(dic_msg['FeedData'][0])=type(res)=list\n",
    new_dic=res[0]#type(new_dic)=dic\n",
    name_list=new_dic['NAMES']#type(new_dic['NAMES'])=list\n",
    value_list=new_dic['VALUES'][0]#type(new_dic['Values'])=list of list\n",
  
    #make one dic from one parsed msg => tag
    plc_dic={}
    for name,value in zip(name_list,value_list):
        #print(name,':',value)
        if value is None:
            value=str(-1)
        plc_dic[key_conversion[name]]=str(value).lower()
        
    #print(plc_dic)
    
    #make primary key for the tag
    pk_tag=plc_dic['line']+'.'+plc_dic['zone']+'.'+plc_dic['station']+'.'+plc_dic['asset']+'.'+plc_dic['tag_name']
    pk_tag=pk_tag.replace('.-1','')
    #print(pk_tag)
    
    plc_tag_msg[pk_tag]=plc_dic
    
    
    #redis write    
    r_val=json.dumps(plc_tag_msg)
    #r.set('plc_tag_hash',r_val)#UNCOMMENT
    
    #redis read
    '''
    r_had=r.get('plc_tag_hash')
    plc_tag=json.loads(r_had)
    #print('from redis read: ',plc_tag)
    #'''    
    
    
    
    #print(plc_dic)
    #print('tag_name: ',plc_dic['tag_name'])
    #print('tag_value: ',plc_dic['tag_value'])
    
    if msg_count%100==0:
        print(msg_count,end=' ')
    if msg_count==30000:
        print('done')
        break
    #break

In [ ]:
from json import loads
from kafka import KafkaConsumer

topic_name = 'magnaPlcTagData'
b_server=['10.0.1.96:9092']

consumer = KafkaConsumer(topic_name, bootstrap_servers=b_server,
                          enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
for message in consumer:
  message = message.value
  print('{}'.format(message))
  break

ModuleNotFoundError: ignored

In [ ]:
!pip install kafka-python
from time import sleep
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer

In [ ]:
from json import loads
consumer = KafkaConsumer(
    'spring_test',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')));

for message in consumer:
  message = message.value;
  print('{}'.format(message))

In [ ]:
topic_name = 'magnaPlcTagData'
consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest', 
                         bootstrap_servers=['10.0.1.96:9092'], api_version=(0, 10), consumer_timeout_ms=1000000)
for msg in consumer:
  print(msg)

In [ ]:
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers='10.0.1.96:9092')
producer.send('sample', b'Hello, World!')
producer.send('sample', key=b'message-two', value=b'This is Kafka-Python')

from kafka import KafkaConsumer
consumer = KafkaConsumer('sample')
for message in consumer:
    print (message)

In [ ]:
if __name__ == '__main__':
    print('Running Consumer..')
    parsed_records = []
    topic_name = 'raw_recipes'
    parsed_topic_name = 'parsed_recipes'

    consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest',
                             bootstrap_servers=['localhost:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    for msg in consumer:
        html = msg.value
        result = parse(html)
        parsed_records.append(result)
    consumer.close()
    sleep(5)

    if len(parsed_records) > 0:
        print('Publishing records..')
        producer = connect_kafka_producer()
        for rec in parsed_records:
            publish_message(producer, parsed_topic_name, 'parsed', rec)

Running Consumer..


In [ ]:
#IP: 10.0.1.96
#Port: 9092
#Topic name: magnaPlcTagData
from time import sleep
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer


if __name__ == '__main__':
    print('Running Consumer..')
    parsed_records = []
    topic_name = 'magnaPlcTagData'
    parsed_topic_name = 'parsed_recipes'

    #consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest', bootstrap_servers=['localhost:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest', bootstrap_servers=['10.0.1.96:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    for msg in consumer:
        html = msg.value
        result = parse(html)
        parsed_records.append(result)
    consumer.close()
    sleep(5)

    if len(parsed_records) > 0:
        print('Publishing records..')
        producer = connect_kafka_producer()
        for rec in parsed_records:
            publish_message(producer, parsed_topic_name, 'parsed', rec)

Running Consumer..


In [ ]:
topic_name = 'magnaPlcTagData'
consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest', 
                         bootstrap_servers=['10.0.1.96:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
for msg in consumer:
  print(msg.key)

In [ ]:
from kafka import KafkaConsumer

# To consume latest messages and auto-commit offsets
consumer = KafkaConsumer('magnaPlcTagData', bootstrap_servers=['10.0.1.96:9092'],api_version=(0, 10, 1))
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    #print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
    #                                     message.offset, message.key,
    #                                      message.value))
    print(msg.key)

In [ ]:
#pycharm code
#import kafka
from kafka import KafkaConsumer
import pandas
topic_name = 'magnaPlcTagData'
consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest',
                         bootstrap_servers=['10.0.1.96:9092'], api_version=(0, 10), consumer_timeout_ms=10)

#consumer = KafkaConsumer(topic_name, bootstrap_servers=['10.0.1.96:9092'])
for msg in consumer:
  print(msg)

# communication_module

In [ ]:
#Mqtt pulls from redis keys

#!pip3 install paho-mqtt
import paho.mqtt.client as mqtt #import the client1
broker_address="localhost"#"192.168.1.184" 
#broker_address="iot.eclipse.org" #use external broker
client = mqtt.Client("P1") #create new instance
client.connect(broker_address) #connect to broker
client.publish("house/main-light","OFF")#publish

In [ ]:
import requests

#url = "https://plc-iot.c2m.net/api/v1/data/process"
#url = "https://nam04.safelinks.protection.outlook.com/?url=https%3A%2F%2Fhelio-ent.c2m.net%2Flogin.aspx&data=02%7C01%7Cayeshas%40okstate.edu%7Ce33dcfb71016446bad6608d7f6b3bb95%7C2a69c91de8494e34a230cdf8b27e1964%7C0%7C1%7C637249124117860660&sdata=%2F6sf3qNRTsDbmWgWdUDKsS%2FrOy6vJ%2FiHgA1ocMWNMbo%3D&reserved=0"
#url = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XipmK93J5vGOb8hvZ/S8rge856C/TG6ovA7DCelU@/Y=&parameterType=LISTDATA"
url = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XipmK93J5vGOb8hvZ/S8rge856C/TG6ovA7DCelU@/Y=&parameterType=LISTDATA"
#headers = {"Authorization": "CSWQpToX4KLCwODDd3QCmBzak8/DYJ" }
response = requests.get(url)#,headers=headers)
print(response.text)




# Standard API format to request data from helio server

In [ ]:
import requests
token_string = ""
url=""
headers= {"Authorization": token_string}
response=requests.get(url,headers=headers)

# Requesting data from PLC feed of helio server

feedID= XipmK93J5vGOb8hvZ/S8rge856C/TG6ovA7DCelU@/Y=<br>
Given Feed ID: iGfXcckbZrSCd8G4xjek1TV9t@ZbzdAP4lU0/bUeyQQ= 

In [ ]:
import requests
url = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XipmK93J5vGOb8hvZ/S8rge856C/TG6ovA7DCelU@/Y=&parameterType=LISTDATA"

#url="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=iGfXcckbZrSCd8G4xjek1TV9t@ZbzdAP4lU0/bUeyQQ=&parameterType=LISTDATA"
#headers = {"Authorization": "CSWQpToX4KLCwODDd3QCmBzak8/DYJ" }
response = requests.get(url)#,headers=headers)
print(response.text)


# Print XML plc tags as string

In [ ]:
import xml.etree.ElementTree as ET

response_text=ET.fromstring(response.text)#prints wordpad with xd 
response_xmlstr = ET.tostring(response_text, encoding='utf8', method='xml')#prints 1 line but has all the info
xml_str = response_xmlstr.decode()#prints wordpad without xd

#print(type(response_text))#<class 'xml.etree.ElementTree.Element'>
#print(type(response_xmlstr))# <class 'bytes'>
#print(type(xml_str))# <class 'str'>
print(xml_str)# <class 'str'>


# Traversing the xml tree of plc tags

In [ ]:
for node in response_text.iter():
  print (node.tag, node.attrib, node.text)
  #print(node)#Elements, such as <Element 'root.EquipmentID' at 0x7f05be878868>, <Element 'root.TagDateTime' at 0x7f05be878598>,<Element 'Table1' at 0x7f05be8785e8>
  #print(node.text)
  #if node.text=='RW43.PLC01.Zone1.ST045.Equip018.CycleTime':
    #print(node.tag)

In [ ]:
#install and import redis
!pip install redis
!pip install t_log
import redis

In [ ]:
#how to setup connection with redis, where do I get password?
# https://opensource.com/article/18/4/how-build-hello-redis-with-python
#next task would be write to redis
#then next task would be read back from redis by reconnecting
import redis

redis_host = "localhost"
redis_port = 6379
redis_password = ""
'''
r = pyredis.Redis(host='localhost', port=redis_port, password=redis_password)
r.set("msg:hello", "Hello Redis!!!")
msg = r.get("msg:hello")
print(msg)
#'''

res = redis.Redis(host='47.99.94.49', port=6379, db=0)
res.set("name","Ayesha")
print(res.get('name'))


In [ ]:
!pip install redis-server
!redis-cli

In [ ]:
!pip install redis

In [ ]:
import redis

# step 2: define our connection information for Redis
# Replaces with your configuration information
redis_host = "192.168.2.11"#"localhost"
redis_port = 6379
redis_password = ""


def hello_redis():
    """Example Hello Redis Program"""
   
    # step 3: create the Redis Connection object
    try:
   
        # The decode_repsonses flag here directs the client to convert the responses from Redis into Python strings
        # using the default encoding utf-8.  This is client specific.
        r = redis.StrictRedis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
   
        # step 4: Set the hello message in Redis
        r.set("msg:hello", "Hello Redis!!!")

        # step 5: Retrieve the hello message from Redis
        msg = r.get("msg:hello")
        print(msg)        
   
    except Exception as e:
        print(e)



hello_redis()

Error 110 connecting to 192.168.2.11:6379. Connection timed out.


In [ ]:
import redis


#res = redis.Redis(host='192.168.2.11', port=6379, db=0)
#res = redis.Redis(host="localhost", port=6379,db=0)#password="")
#r = redis.Redis(host='localhost', port=6379, decode_responses=True)
#r = redis.Redis(host='192.168.2.11', port=6379, decode_responses=True)
rC = redis.Redis(host='192.168.2.11', port=6379, db=0)
#rC = redis.StrictRedis(host='localhost',port=6379,db=0)
rC.set("name","Ayesha")
print(rC.get('name'))


In [ ]:
import the Redis client
import redis


# Create a redis client
redisClient = redis.StrictRedis(host='localhost', port=6379, db=0)

# Add key value pairs to the Redis hash
hashName = "Dessert"
redisClient.hset(hashName, 'line', "Cheesecake")
redisClient.hset(hashName, 'zone', "zone01")
redisClient.hset(hashName, 'station', "-1")
redisClient.hset(hashName, 'asset', "-1")
redisClient.hset(hashName, 'tag_name', "Zone01_Shift_1")
redisClient.hset(hashName, 'tag_value', "1")

# Print the hash
print(redisClient.hgetall(hashName))

# Remove a key
redisClient.hdel(hashName, 1)
 

# Print the hash after removing a key
print(redisClient.hgetall(hashName))

{'line': 'rw43',
 'zone': 'zone01',
 'station': '-1',
 'asset': '-1',
 'tag_name': 'Zone01_Shift_1',
 'tag_value': '1'
} 



In [ ]:
import pickle
import redis

r = redis.StrictRedis('localhost')
mydict = {1:2,2:3,3:4}
p_mydict = pickle.dumps(mydict)
r.set('mydict',p_mydict)

read_dict = r.get('mydict')
yourdict = pickle.loads(read_dict)

In [ ]:
!pip install redis

In [ ]:
import redis
r = redis.Redis(host="192.168.2.11", port=6379, db=0)
x = r.get("analytics_results")
print(x)

<br>

# <font color='blue'> Workflow feeds </font>

# Requesting data from Workflow feed of helio server


In [ ]:
#experiment workflow
# https://stackoverflow.com/questions/44344077/python-requests-get-from-multiple-pages 
import grequests  
urls = ['http://google.com', 'http://yahoo.com', 'http://bing.com']  
unsent_request = (grequests.get(url) for url in urls)

results = grequests.map(unsent_request) 


#https://stackoverflow.com/questions/53773904/python-to-retrieve-multiple-pages-of-data-from-api-with-get 
# ASK AJAY AND KHURSHEED WHY THERE IS NO PAGE INFO
import requests
for page in range(1,4458709):
    url = 'https://api.safecast.org/en-US/measurements.json?page=%s'%page
    data = requests.get(url)
    print data.json()

If two lines can have different shifts (and I am assuming so), then schedule should go under that line<br>
If not, we can have it as a separate key outside the lines <br>
If not, that makes things easier (smile)


Under a job, there is shift1, line etc<br>
Question is: are other shifts and line are without jobs?

In [ ]:
!pip install xmltodict
import requests
#info: data will be overwritten on workflow feed- Ajay confirmed
#looks like to read multiple pages one url is not enough
#


#links for 5 workflow feeds
url_mShift = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=moU39Gbxtz9IhQO0gUy@EQAKbJcR374vb@IVb6kWwH0=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"# 3 record
url_mJob="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GIcqmjbvPvCFX3QNIsJbTNa99MyIUuvAFUyKPTBcs9g=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #has zero records
url_mStation="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GN3jGNVF03thDKOMppdZf/hjLH5vss2SgBlEkNaZvtE=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #20 records
url_mZone="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=LYZEOIUPmsEILn4DoGwhlGw0VaYZ6Yh8@bEKrGOlN6A=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 2 records
url_mLine="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XWPuVgYIhUVEZXRsr6gAFq2BPeTCdlLJxRBpvg07WLM=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 1 record
url_mAsset="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=/y0R6s3pWwReO8a/3ef9tmtO8RAT/U/AKFfmPAD3egA=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"


#get response
url=url_mShift
response = requests.get(url)#,headers=headers)

import xml.etree.ElementTree as ET
response_text=ET.fromstring(response.text)#prints wordpad with xd 
response_xmlstr = ET.tostring(response_text, encoding='utf8', method='xml')#prints 1 line but has all the info
xml_str = response_xmlstr.decode()#prints wordpad without xd


#print response
print(response.text)





<?xml version="1.0" encoding="utf-8"?><root><response status="SUCCESS" startrecord="1" fetchedrecords="6" totalrecords="6"><NewDataSet>&#xD;
  <Table1>&#xD;
    <c2mdatetime>2020-07-17T14:31:49.329Z</c2mdatetime>&#xD;
    <TRNSCTNID>11856</TRNSCTNID>&#xD;
    <CRTDON>2020-06-25T03:08:44.000Z</CRTDON>&#xD;
    <MODFON>2020-06-29T14:01:11.000Z</MODFON>&#xD;
    <CRTDBY>3871</CRTDBY>&#xD;
    <MODFBY>3871</MODFBY>&#xD;
    <fmsscheduleid>1JTE</fmsscheduleid>&#xD;
    <fmsschedulename>Shift 1</fmsschedulename>&#xD;
    <fmsschedulelinelink>RW43</fmsschedulelinelink>&#xD;
    <fmsscheduledays/>&#xD;
    <fmsschedulestarttime>12:00 Am</fmsschedulestarttime>&#xD;
    <fmsscheduleendtime>08:00 Am</fmsscheduleendtime>&#xD;
    <fmsschedulebreakstart>02:00 Am</fmsschedulebreakstart>&#xD;
    <fmsschedulebreakend>02:15 Am</fmsschedulebreakend>&#xD;
    <fmsschedulelunchstart>04:00 Am</fmsschedulelunchstart>&#xD;
    <fmsschedulelunchend>05:00 Am</fmsschedulelunchend>&#xD;
    <fmsscheduledaysnew>

# Print workflow feeds as string

In [ ]:
import xml.etree.ElementTree as ET

response_text=ET.fromstring(response.text)#prints wordpad with xd 
response_xmlstr = ET.tostring(response_text, encoding='utf8', method='xml')#prints 1 line but has all the info
xml_str = response_xmlstr.decode()#prints wordpad without xd

#print(type(response_text))#<class 'xml.etree.ElementTree.Element'>
#print(type(response_xmlstr))# <class 'bytes'>
#print(type(xml_str))# <class 'str'>
print(xml_str)# <class 'str'>


# Parse workflow feed which is an xml tree & put in a dic

In [ ]:
count=0
for node in response_text.iter():
  #print (node.tag, node.attrib, node.text)
  
  #print(node.tag)#Elements, such as <Element 'root.EquipmentID' at 0x7f05be878868>, <Element 'root.TagDateTime' at 0x7f05be878598>,<Element 'Table1' at 0x7f05be8785e8> KEY
  #print(node.tag)
  if node.tag=='response':
    print(node.text)
    dic=node.attrib
    print(node.attrib)
    print(dic['fetchedrecords'])
    print(dic['totalrecords'])
    
  if node.tag=='Table1':
    count=count+1
    #print(node.tag)
    #print(node.text)
print('total',count)
  #print(node.text) #should go to dic value? make that sure VALUE
  #print(node.text)
  #if node.text=='RW43.PLC01.Zone1.ST045.Equip018.CycleTime':
    #print(node.tag)

None
{'status': 'SUCCESS', 'startrecord': '1', 'fetchedrecords': '47', 'totalrecords': '47'}
47
47
total 47


In [ ]:
d={'a':2, 'b':5, 'c':8}

d['aa'] = d.pop('a')
print(d)

{'b': 5, 'c': 8, 'aa': 2}


In [ ]:
#info: data will be overwritten on workflow feed- Ajay confirmed
#looks like to read multiple pages one url is not enough
!pip install xmltodict
from datetime import date, datetime


#links for 6 workflow feeds
url_mShift = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=moU39Gbxtz9IhQO0gUy@EQAKbJcR374vb@IVb6kWwH0=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"# 3 record
url_mJob="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GIcqmjbvPvCFX3QNIsJbTNa99MyIUuvAFUyKPTBcs9g=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #has zero records
url_mStation="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GN3jGNVF03thDKOMppdZf/hjLH5vss2SgBlEkNaZvtE=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #20 records
url_mZone="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=LYZEOIUPmsEILn4DoGwhlGw0VaYZ6Yh8@bEKrGOlN6A=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 2 records
url_mLine="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XWPuVgYIhUVEZXRsr6gAFq2BPeTCdlLJxRBpvg07WLM=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 1 record
url_mAsset="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=/y0R6s3pWwReO8a/3ef9tmtO8RAT/U/AKFfmPAD3egA=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"

url={'asset': url_mAsset, 'shift': url_mShift , 'job': url_mJob, 'station': url_mStation, 'zone': url_mZone, 'line': url_mLine}

keys_to_extract= { 'asset' : ["linelink", "zonelink", "stationlink","assetname", "industry","typeofasset","modelnumber","assetmanufacturer","plancycletime"] ,
                  'station' : ['fmsstationname', 'fmsstationlinelink', 'fmsstationzonelink','fmsstationplannedmnt', 'fmsstationplannedcyctim', 'fmsstationjobsplanned', 'fmsstationtimetoent', 'fmsstationtimetopalm'],
                  'zone' : ['fmszonename', 'fmszonelinelinking', 'fmszoneplannedmnt', 'fmszoneplannedcyctim', 'fmszonejobsplanned'],
                  'shift' : ['fmsschedulename', 'fmsschedulelinelink', 'fmsschedulestarttime', 'fmsscheduleendtime','fmsschedulebreakstart','fmsschedulebreakend','fmsschedulelunchstart','fmsschedulelunchend','fmsscheduledaysnew'],
                  'line' : ['fmslinelinename', 'fmslineplannedcycletime', 'fmslinejobsplanned', 'fmslineplannedmaintenac'],
                  'job' : ['fmsjobname', 'fmsjoblinelink', 'fmsjobsshiftlink', 'fmsjobsstyle' ]}


keys_to_convert={ 'asset' : {'linelink':'line', 'zonelink':'zone', 'stationlink':'station','assetname':'asset', 'industry':'industry','typeofasset':'type','modelnumber':'model_number','assetmanufacturer':'manufacturer','plancycletime':'planned_cycle_time' },
                 

                  'station' : {'fmsstationname':'station', 'fmsstationlinelink':'line', 'fmsstationzonelink':'zone','fmsstationplannedmnt':'next_maintenance', 
                               'fmsstationplannedcyctim':'planned_cycle_time', 'fmsstationjobsplanned':'planned_jobs', 'fmsstationtimetoent':'planned_op_time_to_enter', 'fmsstationtimetopalm':'planned_op_time_to_palm'},
                 
                  'zone' : {'fmszonename':'zone', 'fmszonelinelinking':'line', 'fmszoneplannedmnt':'next_maintenance', 'fmszoneplannedcyctim':'planned_cycle_time', 'fmszonejobsplanned':'planned_jobs'},
                 
                  'shift' : {'fmsschedulename':'shift', 'fmsschedulelinelink':'line',  'fmsschedulestarttime':'shift_start', 'fmsscheduleendtime':'shift_end','fmsschedulebreakstart':'break_start',
                             'fmsschedulebreakend':'break_end','fmsschedulelunchstart':'lunch_start','fmsschedulelunchend':'lunch_end','fmsscheduledaysnew':'days_active'},
                 

                  'line' : {'fmslinelinename':'line', 'fmslineplannedcycletime':'planned_cycle_time', 'fmslinejobsplanned':'planned_jobs', 'fmslineplannedmaintenac':'next_maintenance'},
                 

                  'job' : {'fmsjobname':'job', 'fmsjoblinelink':'line', 'fmsjobsshiftlink':'shift', 'fmsjobsstyle':'style'}}
                 

def convert_keys(a_subset,keys_to_extract_test,create_key,level):
  new_keys=keys_to_convert[level]#new_keys is a dictionary with old_key as key and new_key as value
  #print('new',new_keys)
  #change the key names of the a_subset as suggested in keys_to_convert 
  a_subset=dict([(new_keys.get(k), v) for k, v in a_subset.items()])
  #print(a_subset)
  return a_subset
  
def remove_redundant(a_subset,create_key,rid):
  a_subset_new=a_subset.copy()
  if rid==1:
    key_to_remove_list=create_key[0:-1]#make a list without the last key since last key is the name
    [a_subset_new.pop(key) for key in key_to_remove_list ]#redundant removed
  return a_subset_new


def fix_datetime(a_subset_new):
  
  date_str=a_subset_new['next_maintenance']
  format_str = '%m/%d/%Y' # The format
  datetime_obj = datetime.strptime(date_str, format_str)
  #print(datetime_obj.date())
  
  a_subset_new['next_maintenance']= datetime_obj
  return a_subset_new

def make_shift_tuple(a_subset_new):
    #keys to drop after tuples are made
    key_to_remove_listy=['shift_start','shift_end','break_start','break_end','lunch_start','lunch_end']
    format_str='%Y-%m-%d %H:%M %p'
    str1='%Y-%m-%d %H:%M %S'
    str2='%Y-%m-%d %I:%M %S %p'
    for ky in key_to_remove_listy:
        #time_str=a_subset_new[ky]#+str(datetime.date(datetime.now()))
        time_str=str(datetime.date(datetime.now()))+' ' +a_subset_new[ky]
        
        #a_subset_new[ky] = datetime.strptime(time_str, format_str)
        a_subset_new[ky] = datetime.strptime(time_str, format_str).strftime(str2)
        
    #make tuple of schedule
    a_subset_new['shift_schedule']=(a_subset_new['shift_start'],a_subset_new['shift_end'])
    print(a_subset_new['shift_schedule'][0])

    #make tuple of break
    a_subset_new['break']=(a_subset_new['break_start'],a_subset_new['break_end'])

    #make tuple of lunch
    a_subset_new['lunch']=(a_subset_new['lunch_start'],a_subset_new['lunch_end'])

    #drop keys
    [a_subset_new.pop(key) for key in key_to_remove_listy ]#redundant removed

    return a_subset_new



def replace_url(i,url_test):#don't need this
  page_number=url_test[-10:].split('=')[1]
  replace_this_str="startrecord="+page_number
  by_this_str="startrecord="+str(i)
  url_test=url_test.replace(replace_this_str,by_this_str)
  
  return url_test

#test replace_url
'''
url['asset']=replace_url(20, url['asset'])
url['asset']=replace_url(3, url['asset'])
for i in range(leng_asset):#how to get to next page
  url_a=replace_url(i,url_a)#replace the url with new page 
  response = requests.get(url_a)#get xml tree of assets

#'''

"\nurl['asset']=replace_url(20, url['asset'])\nurl['asset']=replace_url(3, url['asset'])\nfor i in range(leng_asset):#how to get to next page\n  url_a=replace_url(i,url_a)#replace the url with new page \n  response = requests.get(url_a)#get xml tree of assets\n\n#"

In [ ]:
import requests
import xml.etree.ElementTree as ET
import math
import requests
import xmltodict
import json
from collections import OrderedDict
from datetime import date, datetime




rid=1#remove redundant keys

def get_all(url_level,keys_to_extract_test,level,create_key,dicty):
  response = requests.get(url_level)#(url['asset'])#,headers=headers
  xml=response.text
  workflow_dict=xmltodict.parse(xml)
  new_keys=keys_to_convert[level]
  create_key = [new_keys[x] for x in create_key]#convert the key using new keys defined in keys_to_convert

  for key, value in workflow_dict.items():#looks like 1 key 1 value
    #print(key)#prints root

    for key2, value2 in value.items():
      #print(key2)#prints response
      dic_outer=json.loads(json.dumps(value2))
      fetched_records=dic_outer['@fetchedrecords']
      total_records=dic_outer['@totalrecords']
      pages=math.ceil(int(total_records)/int(fetched_records))


      dataset_value=dic_outer['NewDataSet']
      total_dics=len(dataset_value['Table1'])#dataset_value['Table1'] ===> it's a list
      print('total_records',total_dics)

      for i in range(total_dics):#total_dics == total_records

        dict_i=dataset_value['Table1'][i] #type(dataset_value['Table1'][0])====>it's a dict
        #print(dict_i)

        a_subset = {keyu: dict_i[keyu] for keyu in keys_to_extract_test} #extract necessary key from asset dictionary, keys_to_extract_asset
        a_subset = dict((k, str(v).lower()) for k, v in a_subset .items()) #values to lower case
        a_subset = convert_keys(a_subset,keys_to_extract_test, create_key, level)


        #a_subset=convert_keys(a_subset,keys_to_extract_test,level)#change the key names
        if level=='line':
          a_subset=fix_datetime(a_subset)   
          dicty[a_subset[create_key[0]]]=a_subset
        
        if level=='zone':
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          a_subset_new=fix_datetime(a_subset_new)                    
          if 'zones' not in dicty[a_subset[create_key[0]]]:
            dicty[a_subset[create_key[0]]]['zones']={a_subset[create_key[1]]:a_subset_new}
          else:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]=a_subset_new


        if level=='shift':
          a_subset[create_key[1]]= a_subset[create_key[1]].replace(" ","_")#remove space from shift: shift 1==> shift_1
          a_subset=make_shift_tuple(a_subset)
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          
          if 'shifts' not in dicty[a_subset[create_key[0]]]:
            dicty[a_subset[create_key[0]]]['shifts']={a_subset[create_key[1]]:a_subset_new}
          else:
            dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]=a_subset_new
           

        if level=='job':
          a_subset[create_key[1]]= a_subset[create_key[1]].replace(" ","_")#remove space from shift: shift 1==> shift_1
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          if 'jobs' not in dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]:
            dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]['jobs']={a_subset[create_key[2]]:a_subset_new}

          else:
            dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]['jobs'][a_subset[create_key[2]]]=a_subset_new



      
        if level=='station':
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          a_subset_new=fix_datetime(a_subset_new)   
          if 'stations' not in dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations']={a_subset[create_key[2]]:a_subset_new}

          else:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]=a_subset_new



        if level=='asset':
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          if 'assets' not in dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]['assets']={a_subset[create_key[3]]:a_subset_new}
          else:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]['assets'][a_subset[create_key[3]]]=a_subset_new
          

      
      return dicty

      
level='line'
line_key=['fmslinelinename']
dicty={}
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,line_key,dicty)
#print(dicty)



level='shift'
shift_key=['fmsschedulelinelink','fmsschedulename' ]
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,shift_key,dicty)
#print(dicty)


level='job'
job_key=['fmsjoblinelink', 'fmsjobsshiftlink', 'fmsjobname']#line+shift+job
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,job_key,dicty)#change key name
#print(dicty)



level='zone'
zone_key=['fmszonelinelinking','fmszonename']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,zone_key,dicty)
#print(dicty)


level='station'
station_key=['fmsstationlinelink', 'fmsstationzonelink', 'fmsstationname']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,station_key,dicty)
#print(dicty)

level='asset'
asset_key=['linelink', 'zonelink', 'stationlink','assetname']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,asset_key,dicty)
#print(dicty)
#'''

print (json.dumps(dicty, indent=1, default=str))
#'''





total_records 2
total_records 6
helo done
helo done
helo done
helo done
helo done
helo done
total_records 2
total_records 4
total_records 40
total_records 47
{
 "rw44": {
  "line": "rw44",
  "planned_cycle_time": "300",
  "planned_jobs": "10",
  "next_maintenance": "2020-07-31 00:00:00",
  "shifts": {
   "shift_1": {
    "shift": "shift_1",
    "days_active": "monday!tuesday!wednesday!thursday!friday",
    "shift_schedule": [
     "1900-01-01 12:00:00",
     "1900-01-01 08:00:00"
    ],
    "break": [
     "1900-01-01 02:00:00",
     "1900-01-01 02:15:00"
    ],
    "lunch": [
     "1900-01-01 04:00:00",
     "1900-01-01 05:00:00"
    ],
    "jobs": {
     "gm001": {
      "job": "gm001",
      "style": "none"
     }
    }
   },
   "shift_2": {
    "shift": "shift_2",
    "days_active": "monday!tuesday!wednesday!thursday!friday",
    "shift_schedule": [
     "1900-01-01 08:00:00",
     "1900-01-01 04:00:00"
    ],
    "break": [
     "1900-01-01 10:00:00",
     "1900-01-01 10:30:00"
  

In [ ]:
#redis host
redis_host="192.168.2.11"
redis_port = 6379
redis_password = ""
r=redis.Redis(host="192.168.2.11", port=6379, db=0)


#write workflowfeeds on redis
r_val=json.dumps(dicty)
r.set('workflow_lines',r_val)#UNCOMMENT
r.get('workflow_lines')

#new link: https://colab.research.google.com/drive/1NkMcpPuWcTBJ1_69DQPtOaO17dVIuANW#scrollTo=mK68snSLJ01C 

In [ ]:
#without dic
import requests
import xml.etree.ElementTree as ET
import math
import requests
import xmltodict
import json
from collections import OrderedDict


def get_all(url_level,keys_to_extract_test,level):
  response = requests.get(url_level)#(url['asset'])#,headers=headers
  xml=response.text
  workflow_dict=xmltodict.parse(xml)

  for key, value in workflow_dict.items():#looks like 1 key 1 value
    #print(key)#prints root

    for key2, value2 in value.items():
      #print(key2)#prints response
      dic_outer=json.loads(json.dumps(value2))
      fetched_records=dic_outer['@fetchedrecords']
      total_records=dic_outer['@totalrecords']
      pages=math.ceil(int(total_records)/int(fetched_records))


      dataset_value=dic_outer['NewDataSet']
      total_dics=len(dataset_value['Table1'])#dataset_value['Table1'] ===> it's a list
      print('total_records',total_dics)

      for i in range(total_dics):#total_dics == total_records

        dict_i=dataset_value['Table1'][i] #type(dataset_value['Table1'][0])====>it's a dict
        #print(dict_i)

        a_subset = {keyu: dict_i[keyu] for keyu in keys_to_extract_test} #extract necessary key from asset dictionary, keys_to_extract_asset


        #a_subset=convert_keys(a_subset,keys_to_extract_test,level)#change the key names

        #dicty[a_subset[create_key]]=a_subset
        print(a_subset)
      
      #return dicty

level='shift'
keys_extract=keys_to_extract[level]
get_all(url[level],keys_extract,level)



total_records 6
{'fmsschedulename': 'Shift 2', 'fmsschedulelinelink': 'RW43', 'fmsschedulestarttime': '08:00 Am', 'fmsscheduleendtime': '04:00 Pm', 'fmsschedulebreakstart': '10:00 Am', 'fmsschedulebreakend': '10:15 Am', 'fmsschedulelunchstart': '12:00 Pm', 'fmsschedulelunchend': '01:00 Pm', 'fmsscheduledaysnew': 'Monday!Tuesday!Wednesday!Thursday!Friday'}
{'fmsschedulename': 'Shift 3', 'fmsschedulelinelink': 'RW43', 'fmsschedulestarttime': '04:00 Pm', 'fmsscheduleendtime': '12:00 Am', 'fmsschedulebreakstart': '06:00 Am', 'fmsschedulebreakend': '06:15 Am', 'fmsschedulelunchstart': '08:00 Pm', 'fmsschedulelunchend': '09:00 Pm', 'fmsscheduledaysnew': 'Monday!Tuesday!Wednesday!Thursday!Friday'}
{'fmsschedulename': 'Shift 1', 'fmsschedulelinelink': 'RW44', 'fmsschedulestarttime': '12:00 Am', 'fmsscheduleendtime': '08:00 Am', 'fmsschedulebreakstart': '02:00 Am', 'fmsschedulebreakend': '02:15 Am', 'fmsschedulelunchstart': '04:00 Am', 'fmsschedulelunchend': '05:00 Am', 'fmsscheduledaysnew': 'M

In [ ]:
#loop to make big dictionary

level='line'
keys_extract=keys_to_extract[level]
get_all(url[level],keys_extract,level)




total_records 2
{'fmslinelinename': 'RW43', 'fmslineplannedcycletime': '248', 'fmslinejobsplanned': '10', 'fmslineselection': None, 'fmslineplannedmntnc': None, 'fmslinepdm': None, 'fmslineplannedmaintenac': '07/31/2020'}
{'fmslinelinename': 'RW44', 'fmslineplannedcycletime': '300', 'fmslinejobsplanned': '10', 'fmslineselection': None, 'fmslineplannedmntnc': None, 'fmslinepdm': None, 'fmslineplannedmaintenac': '07/31/2020'}


In [ ]:
#a_subset['shift']= a_subset['shift'].replace(" ","_")
'shift 1'.replace(" ","_")

'shift_1'

In [ ]:
import requests
import json
import xmltodict
import json
from collections import OrderedDict

response = requests.get(url['asset'])#,headers=headers
xml=response.text
my_dict=xmltodict.parse(xml)




for key,value in my_dict.items():
  print(key)
  for key2,value2 in value.items():
    print(key2)
    dic_outer=json.loads(json.dumps(value2))
    print(dic_outer)
    print(dic_outer['@fetchedrecords'])
    print(dic_outer['@totalrecords'])

    dataset_value=dic_outer['NewDataSet']
    total_dics=len(dataset_value['Table1'])#dataset_value['Table1'] ===> it's a list
    print('total_records',total_dics)
    dict_i=dataset_value['Table1'][0] #type(dataset_value['Table1'][0])====>it's a dict
    print(dict_i)
    
    keys_to_extract_asset = ["assetname", "industry","typeofasset","modelnumber","assetmanufacturer","plancycletime"]

    a_subset = {keyu: dict_i[keyu] for keyu in keys_to_extract_asset} #extract necessary key from asset dictionary
    print(a_subset)
    



root
response
{'@status': 'SUCCESS', '@startrecord': '1', '@fetchedrecords': '47', '@totalrecords': '47', 'NewDataSet': {'Table1': [{'c2mdatetime': '2020-07-02T17:35:33.466Z', 'trnsctnid': '11897', 'assetid': '5JOK7', 'assetname': '300SR1', 'industry': 'Manufacturing', 'typeofasset': 'Robotic Arm', 'modelnumber': '210 F', 'assetmanufacturer': 'Fanuc', 'assetyear': '2020', 'assetapplication': None, 'assetoperation': 'Automatic', 'Monitoringdeviceid': None, 'CreatededOn': '2020-06-30T23:14:20.000Z', 'ModifiedOn': '2020-06-30T23:14:20.000Z', 'StageName': 'Asset Status', 'StateName': 'Active', 'username': 'lmv_admin@yopmail.com', 'companyname': 'MagnaLMVCompany4', 'groupname': 'MagnaLMVCompany4', 'region': 'North', 'tco': '1000', 'linelink': 'RW44', 'stationlink': 'ST300', 'zonelink': 'Zone02', 'plancycletime': '5'}, {'c2mdatetime': '2020-07-02T17:35:33.466Z', 'trnsctnid': '11898', 'assetid': '5SE9N', 'assetname': '300MR2', 'industry': 'Manufacturing', 'typeofasset': 'Robotic Arm', 'modeln

In [ ]:
listy=['b','d']
d={'a':'aa','b':'bb','c':'cc','d':'dd'}
[d.pop(key) for key in listy ]
print(d)

{'a': 'aa', 'c': 'cc'}


In [ ]:
#STore
# HOW TO FETCH RECORD BY RECORD: IS IT POSSIBLE IN MY CASE?
# https://stackoverflow.com/questions/45293458/how-to-read-the-next-page-on-api-using-python


def get_records(response_text):
    for node in response_text.iter():
      if node.tag=='response':
        print(node.attrib)
        fetched_records=node.attrib['fetchedrecords']# node.attrib ==> dictionary of response tag in asset level
        total_records=node.attrib['totalrecords']
        #print(fetched_records)
        break

    leng=math.ceil(int(total_records)/int(fetched_records))
    return leng, fetched_records, total_records


response = requests.get(url['asset'])#,headers=headers
response_text=ET.fromstring(response.text)#prints wordpad with xd 
leng_asset, fetched_records, total_records=get_records(response_text)
print(fetched_records, total_records)

